In [20]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# =====================================================
# STEP 1 — LOAD DATA
# =====================================================

train_path = "/content/drive/MyDrive/ML_Project/MultinomialClassification/Dataset/train.csv"
test_path  = "/content/drive/MyDrive/ML_Project/MultinomialClassification/Dataset/test.csv"



df_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)

TARGET = "spend_category"
ID_COL = "trip_id"

print("Original Training Shape:", df_raw.shape)
print("Original Test Shape:", test_raw.shape)


# =====================================================
# HELPER — Convert Range (15-30, 90+) to numeric
# =====================================================
def range_to_mid(x):
    x = str(x).strip()
    if x.lower() in ["none", "", "nan", "null"]:
        return np.nan
    if "+" in x:
        return float(x.replace("+", ""))
    if "-" in x:
        a, b = x.split("-")
        return (float(a) + float(b)) / 2
    try:
        return float(x)
    except:
        return np.nan


# =====================================================
# STEP 2 — GLOBAL COLUMN DEFINITIONS
# =====================================================
binary_cols = [
    "is_first_visit","intl_transport_included","accomodation_included",
    "food_included","domestic_transport_included","sightseeing_included",
    "guide_included","insurance_included"
]

categorical_cols = [
    "country","age_group","travel_companions","main_activity",
    "visit_purpose","tour_type","info_source","arrival_weather"
]

numeric_count_cols = ["num_females","num_males","mainland_stay_nights","island_stay_nights"]


# =====================================================
# STEP 3 — REMOVE NULL TARGETS FIRST
# =====================================================
removed_target_nulls = df_raw[TARGET].isnull().sum()
print("Rows removed due to null spend_category:", removed_target_nulls)

df_raw = df_raw[df_raw[TARGET].notnull()].reset_index(drop=True)
print("Training shape after removing null targets:", df_raw.shape)


# =====================================================
# STEP 4 — MAIN PREPROCESSING FUNCTION
# =====================================================
def preprocess_raw_df(df):
    df = df.copy()

    # Clean strings
    for c in df.columns:
        if df[c].dtype == object:
            df[c] = df[c].astype(str).str.strip().str.rstrip(',')

    # Binary processing
    for c in binary_cols:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip().str.lower()
            df[c] = df[c].replace({
                "yes": 1,
                "no": 0,
                "nan": np.nan,
                "none": np.nan,
                "null": np.nan,
                "": np.nan
            })
            df[c] = df[c].fillna(0).astype(int)

    # Numeric count fields
    for c in numeric_count_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    # =====================================================
    # SAFE ORDINAL ENCODING FOR RANGE COLUMNS
    # =====================================================

    # Clean weird string values
    def clean_str(x):
        x = str(x).strip().lower()
        if x in ["nan", "none", "null", ""]:
            return np.nan
        return x

    # Clean raw string columns
    if "days_booked_before_trip" in df.columns:
        df["days_booked_before_trip_clean"] = df["days_booked_before_trip"].apply(clean_str)

    if "total_trip_days" in df.columns:
        df["total_trip_days_clean"] = df["total_trip_days"].apply(clean_str)

    # Define ordinal mappings
    ordinal_days_booked = {
        "1-7": 1,
        "8-14": 2,
        "15-30": 3,
        "31-60": 4,
        "61-90": 5,
        "90+": 6
    }

    ordinal_total_trip = {
        "1-6": 1,
        "7-14": 2,
        "15-30": 3,
        "30+": 4
    }

    # Map → Fill Missing → Convert to int
    if "days_booked_before_trip_clean" in df.columns:
        df["days_booked_before_trip_ord"] = (
            df["days_booked_before_trip_clean"]
                .map(ordinal_days_booked)
        )

        # Fill NaN with mode **of ordinal values**
        df["days_booked_before_trip_ord"].fillna(
            df["days_booked_before_trip_ord"].mode()[0],
            inplace=True
        )

        df["days_booked_before_trip_ord"] = df["days_booked_before_trip_ord"].astype(int)

    if "total_trip_days_clean" in df.columns:
        df["total_trip_days_ord"] = (
            df["total_trip_days_clean"]
                .map(ordinal_total_trip)
        )

        # Fill NaN with mode of ordinal values
        df["total_trip_days_ord"].fillna(
            df["total_trip_days_ord"].mode()[0],
            inplace=True
        )

        df["total_trip_days_ord"] = df["total_trip_days_ord"].astype(int)



    # Special requirements → binary
    if "has_special_requirements" in df.columns:
        df["has_special_req_bin"] = df["has_special_requirements"].astype(str).apply(
            lambda x: 0 if x.lower() in ["none", "", "nan"] else 1
        )

    return df


# =====================================================
# APPLY PREPROCESSING TO TRAIN & TEST
# =====================================================
df = preprocess_raw_df(df_raw).reset_index(drop=True)
test_df = preprocess_raw_df(test_raw).reset_index(drop=True)

print("\nAfter Base Preprocessing:")
print(df.shape)


# =====================================================
# STEP 5 — IMPUTATIONS (NO NaNs must remain)
# =====================================================

# Categorical mode fill
for c in categorical_cols:
    if c in df.columns:
        mode = df[c].mode()[0]
        df[c] = df[c].fillna(mode)
        test_df[c] = test_df[c].fillna(mode)

# =========================================================
# OUTLIER REMOVAL (TRAIN ONLY)
# And count number of rows removed per condition
# =========================================================

clean_df = df.copy()
initial_rows = len(clean_df)

outlier_info = {}

# ----- num_females ≤ 14 -----
before = len(clean_df)
clean_df = clean_df[clean_df["num_females"] <= 10]
after = len(clean_df)
outlier_info["num_females"] = before - after

# ----- Rule 1: num_males ≤ 13 -----
before = len(clean_df)
clean_df = clean_df[clean_df["num_males"] <= 10]
after_rule1 = len(clean_df)
removed_rule1 = before - after_rule1
before_rule2 = len(clean_df)

# Save results
outlier_info["num_males_threshold"] = removed_rule1
print("Removed (num_males > 20):", removed_rule1)


# ----- mainland_stay_nights ≤ 100 -----
before = len(clean_df)
clean_df = clean_df[clean_df["mainland_stay_nights"] <= 90]
after = len(clean_df)
outlier_info["mainland_stay_nights"] = before - after

# ----- island_stay_nights ≤ 21 -----
before = len(clean_df)
clean_df = clean_df[clean_df["island_stay_nights"] <= 60]
after = len(clean_df)
outlier_info["island_stay_nights"] = before - after

final_rows = len(clean_df)

# =========================================================
# PRINT OUTLIER REMOVAL SUMMARY
# =========================================================
print("\n========== OUTLIER REMOVAL SUMMARY ==========")
for col, removed in outlier_info.items():
    print(f"{col}: removed {removed} rows")

print("---------------------------------------------")
print(f"Total rows removed: {initial_rows - final_rows}")
print(f"Final Training Shape after Outlier Removal: {clean_df.shape}")
print("Test Shape (unchanged):", test_df.shape)


# =====================================================
# STEP 5 — FINAL FEATURE DEFINITIONS (No NaNs left)
# =====================================================

numeric_features = [
    "num_females",
    "num_males",
    "mainland_stay_nights",
    "island_stay_nights",
    "days_booked_before_trip_ord",
    "total_trip_days_ord"
]

binary_features = binary_cols + ["has_special_req_bin"]

categorical_features = categorical_cols

all_features = numeric_features + binary_features + categorical_features

clean_df = clean_df[all_features + [TARGET]]
test_df_final = test_df[all_features]




Original Training Shape: (12654, 25)
Original Test Shape: (5852, 24)
Rows removed due to null spend_category: 34
Training shape after removing null targets: (12620, 25)


/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace({
/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace({
/tmp/ipython-input-4121578799.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] =


After Base Preprocessing:
(12620, 30)
Removed (num_males > 20): 8

========== OUTLIER REMOVAL SUMMARY ==========
num_females: removed 28 rows
num_males_threshold: removed 8 rows
mainland_stay_nights: removed 27 rows
island_stay_nights: removed 8 rows
---------------------------------------------
Total rows removed: 71
Final Training Shape after Outlier Removal: (12549, 30)
Test Shape (unchanged): (5852, 29)


In [21]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

# =====================================================
# STEP X — FEATURE ENGINEERING: K-Means + GMM CLUSTERS
# =====================================================

print("\n=== Running Clustering Feature Engineering (KMeans) ===")

# ----------------------------
# 1. Prepare numeric data only
# ----------------------------
num_only = clean_df[numeric_features].copy()
test_num_only = test_df_final[numeric_features].copy()

# Standardize for clustering
scaler = StandardScaler()
num_scaled = scaler.fit_transform(num_only)
test_num_scaled = scaler.transform(test_num_only)

# ----------------------------
# 2. K-Means Clustering
# ----------------------------
kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
clean_df["kmeans_cluster"] = kmeans.fit_predict(num_scaled)
test_df_final["kmeans_cluster"] = kmeans.predict(test_num_scaled)

# # ----------------------------
# # 3. Gaussian Mixture Model (GMM)
# # ----------------------------
# gmm = GaussianMixture(n_components=5, random_state=42)
# clean_df["gmm_cluster"] = gmm.fit_predict(num_scaled)
# test_df_final["gmm_cluster"] = gmm.predict(test_num_scaled)

print("Clustering features added: kmeans_cluster")

# Add to categorical features
categorical_features += ["kmeans_cluster"]

# Update full feature list
all_features = numeric_features + binary_features + categorical_features

# Important: keep only final columns
clean_df = clean_df[all_features + [TARGET]]
test_df_final = test_df_final[all_features]

print("\nFinal shape after clustering:", clean_df.shape)



=== Running Clustering Feature Engineering (KMeans) ===
Clustering features added: kmeans_cluster

Final shape after clustering: (12549, 25)


/tmp/ipython-input-1854590008.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_final["kmeans_cluster"] = kmeans.predict(test_num_scaled)


In [22]:
preprocess = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ("bin", "passthrough", binary_features)
])

In [23]:
# =====================================================
# CORRECT SPLITTING ACCORDING TO YOUR REQUIREMENT
# =====================================================

from sklearn.model_selection import train_test_split

# ----------- FULL DATASET -----------
full_df = clean_df.copy()

# =====================================================
# SPLIT A: 80% train, 10% val, 10% test
# =====================================================

# First: 80% train, 20% temp
train_80, temp_20 = train_test_split(
    full_df,
    test_size=0.20,
    stratify=full_df[TARGET],
    random_state=42
)

# Now split 20% temp → 10% val + 10% test
val_10, test_10 = train_test_split(
    temp_20,
    test_size=0.50,   # half of 20%
    stratify=temp_20[TARGET],
    random_state=42
)

print("Split A (80/10/10):")
print("train_80:", train_80.shape)
print("val_10:", val_10.shape)
print("test_10:", test_10.shape)


# =====================================================
# SPLIT B: 20% train, 40% val, 40% test
# =====================================================

# First: 20% train, 80% temp
train_20, temp_80 = train_test_split(
    full_df,
    test_size=0.80,
    stratify=full_df[TARGET],
    random_state=42
)

# Now split 80% temp → 40% val + 40% test
val_40, test_40 = train_test_split(
    temp_80,
    test_size=0.50,
    stratify=temp_80[TARGET],
    random_state=42
)

print("\nSplit B (20/40/40):")
print("train_20:", train_20.shape)
print("val_40:", val_40.shape)
print("test_40:", test_40.shape)


Split A (80/10/10):
train_80: (10039, 25)
val_10: (1255, 25)
test_10: (1255, 25)

Split B (20/40/40):
train_20: (2509, 25)
val_40: (5020, 25)
test_40: (5020, 25)


In [24]:

# =====================================================
# STEP 6 — FUNCTION TO MAKE SKEWED AND NON-SKEWED
# =====================================================
def make_skewed(df):
    # keep natural distribution
    return df.copy()

def make_nonskewed(df):
    # upsample minority classes
    major_size = df[TARGET].value_counts().max()
    frames = []
    for cls in df[TARGET].unique():
        cls_df = df[df[TARGET]==cls]
        cls_up = resample(cls_df, replace=True, n_samples=major_size, random_state=42)
        frames.append(cls_up)
    return pd.concat(frames)




In [25]:
from sklearn.neural_network import MLPClassifier

MLP_MODELS = {
    "MLP_small":  MLPClassifier(hidden_layer_sizes=(64,),
                                activation='relu', solver='adam',
                                max_iter=300, random_state=42),

    "MLP_medium": MLPClassifier(hidden_layer_sizes=(128, 64),
                                activation='relu', solver='adam',
                                max_iter=400, random_state=42),

    "MLP_large":  MLPClassifier(hidden_layer_sizes=(256, 128, 64),
                                activation='relu', solver='adam',
                                max_iter=500, random_state=42)
}


In [26]:
# BASE = "/content/drive/MyDrive/ML_Project/MultinomialClassification/SVM/SVM_Tuning"
# os.makedirs(BASE, exist_ok=True)

tasks = [
    ("80_skewed",    make_skewed(train_80),    val_10, test_10),
    ("80_nonskewed", make_nonskewed(train_80), val_10, test_10),

    ("20_skewed",    make_skewed(train_20),    val_40, test_40),
    ("20_nonskewed", make_nonskewed(train_20), val_40, test_40),
]



In [27]:
# =====================================================
# 3) SAVE RESULTS
# =====================================================
def save_mlp_outputs(out_dir, pipe, model_type, mode_name,
                     X_val, y_val, X_test, y_test, X_final):

    os.makedirs(out_dir, exist_ok=True)

    val_pred = pipe.predict(X_val)
    test_pred = pipe.predict(X_test)
    final_pred = pipe.predict(X_final)

    val_acc = accuracy_score(y_val, val_pred)
    test_acc = accuracy_score(y_test, test_pred)

    # Validation file
    with open(f"{out_dir}/{model_type}_{mode_name}_validation.txt", "w") as f:
        f.write(classification_report(y_val, val_pred))
        f.write(f"\nValidation Accuracy: {val_acc}\n")

    # Test file
    with open(f"{out_dir}/{model_type}_{mode_name}_test.txt", "w") as f:
        f.write(classification_report(y_test, test_pred))
        f.write(f"\nTest Accuracy: {test_acc}\n")

    # Summary file
    with open(f"{out_dir}/{model_type}_{mode_name}_summary.txt", "w") as f:
        f.write(f"Validation Accuracy: {val_acc}\n")
        f.write(f"Test Accuracy: {test_acc}\n")

    # Final Kaggle prediction
    submission = pd.DataFrame({
        ID_COL: test_df[ID_COL],
        TARGET: final_pred.astype(int)
    })
    submission.to_csv(f"{out_dir}/{model_type}_{mode_name}.csv", index=False)


In [28]:
# =====================================================
# 4) RUN ALL MLP MODELS
# =====================================================
def run_all_mlps():

    BASE = "/content/drive/MyDrive/ML_Project/MultinomialClassification/NN/MLP(VAL&TRAIN)"
    os.makedirs(BASE, exist_ok=True)

    mlp_sizes = {
        "mlp_small":  (50,),
        "mlp_medium": (100, 50),
        "mlp_large":  (150, 100, 50)
    }

    for model_type, hidden_layers in mlp_sizes.items():

        print("\n============================")
        print(f" Running: {model_type}")
        print("============================\n")

        for mode_name, tr_df, val_df, test_df_int in tasks:

            print(f"---- MODE: {mode_name} ----")

            # Feature splits
            X_train = tr_df[all_features]
            y_train = tr_df[TARGET]

            X_val = val_df[all_features]
            y_val = val_df[TARGET]

            X_test = test_df_int[all_features]
            y_test = test_df_int[TARGET]

            X_final = test_df_final[all_features]

            # Build MLP
            mlp_model = MLPClassifier(
                hidden_layer_sizes=hidden_layers,
                activation="relu",
                solver="adam",
                max_iter=500,
                random_state=42
            )

            pipe = Pipeline([
                ("prep", preprocess),
                ("mlp", mlp_model)
            ])

            # Train
            pipe.fit(X_train, y_train)

            # Save files
            out_dir = f"{BASE}/{model_type}/{mode_name}"
            save_mlp_outputs(
                out_dir, pipe,
                model_type=model_type,
                mode_name=mode_name,
                X_val=X_val, y_val=y_val,
                X_test=X_test, y_test=y_test,
                X_final=X_final
            )

    print("\nAll MLP Models Completed Successfully!\n")


# =====================================================
# RUN EVERYTHING
# =====================================================
run_all_mlps()


 Running: mlp_small

---- MODE: 80_skewed ----


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


---- MODE: 80_nonskewed ----


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


---- MODE: 20_skewed ----


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


---- MODE: 20_nonskewed ----

 Running: mlp_medium

---- MODE: 80_skewed ----
---- MODE: 80_nonskewed ----
---- MODE: 20_skewed ----
---- MODE: 20_nonskewed ----

 Running: mlp_large

---- MODE: 80_skewed ----
---- MODE: 80_nonskewed ----
---- MODE: 20_skewed ----
---- MODE: 20_nonskewed ----

All MLP Models Completed Successfully!

